# GSEA of MAST DEGs against Reactome Pathways

Similar to the analysis for Hallmark Pathways, we'll also perform enrichment tests against Reactome Pathways, which are much more mechanism-focused. These pathways are also hierarchically related, which is important for interpretation of enrichment results. We'll assemble both the gene sets and their relationships for later visualization so that we don't misinterpret related gene sets.

## Load packages

hise: The Human Immune System Explorer R SDK package  
purrr: Functional programming tools  
dplyr: Dataframe handling functions  
tibble: modern data.frame structures  
fgsea: Fast Gene Set Enrichment Analysis  

In [1]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }
quiet_library(hise)
quiet_library(purrr)
quiet_library(dplyr)
quiet_library(tibble)
quiet_library(fgsea)

## Download Reactome gene sets

We'll obtain the Reactome pathways and relationships provided by reactome.org

In [2]:
# Pathway identifiers, names, and species
download.file(
    "https://reactome.org/download/current/ReactomePathways.txt", 
    "ReactomePathways.txt"
)
# Pathway gene sets
download.file(
    "https://reactome.org/download/current/ReactomePathways.gmt.zip", 
    "ReactomePathways.gmt.zip"
)
system("unzip ReactomePathways.gmt.zip")
# Pathway relationships
download.file(
    "https://reactome.org/download/current/ReactomePathwaysRelation.txt",
    "ReactomePathwaysRelation.txt"
)

## Read, filter, and structure gene sets and relationships

### Read and convert .gmt

The gene set GMT file contains one gene set per line, with the set name and id followed by the list of genes.

We'll read these lines, split on tabs, and then use the split data to build a tibble in which each row has a name, id, and gene list.

In [3]:
sets <- readLines("ReactomePathways.gmt")
sets <- strsplit(sets, split = "\t")

In [4]:
sets <- map_dfr(
    sets,
    function(v) {
        tibble(
            name = map_chr(sets, 1),
            id = map_chr(sets, 2),
            genes = lapply(sets, function(x) {x[-c(1,2)]})
        )
})

In [5]:
sets <- sets %>%
  select(id, genes) %>%
  unique()

In [6]:
nrow(sets)

[1] 2612

### Read set IDs and filter for human pathways

Next, we'll read the ReactomePathways file, and use the 3rd column to filter for gene sets from *Homo sapiens*.

In [7]:
pw <- readLines("ReactomePathways.txt")
pw <- strsplit(pw, split = "\t")

In [8]:
pw <- map(pw, as.list)
pw <- map(pw, function(l) { names(l) <- c("id", "name", "species"); l })
pw <- map_dfr(pw, as.data.frame)

In [9]:
head(pw)

,id,name,species
,<chr>,<chr>,<chr>
1,R-BTA-73843,5-Phosphoribose 1-diphosphate biosynthesis,Bos taurus
2,R-BTA-1971475,A tetrasaccharide linker sequence is required for GAG synthesis,Bos taurus
3,R-BTA-1369062,ABC transporters in lipid homeostasis,Bos taurus
4,R-BTA-382556,ABC-family proteins mediated transport,Bos taurus
5,R-BTA-9033807,ABO blood group biosynthesis,Bos taurus
6,R-BTA-418592,ADP signalling through P2Y purinoceptor 1,Bos taurus


In [10]:
pw <- pw %>%
  filter(species == "Homo sapiens") %>%
  select(id, name)

In [11]:
nrow(pw)

[1] 2629

### Load relationships

The last piece required is the relationships between pathways, which we'll structure as a data.frame.

In [12]:
links <- read.table("ReactomePathwaysRelation.txt", sep = "\t")
names(links) <- c("from", "to")
links <- links %>%
  filter(from %in% pw$id)

In [13]:
nrow(links)

[1] 2647

### Identify root nodes

To find the major pathway root nodes and links, we'll select pathways that link *to* other nodes, but don't have links *from* any parent nodes.

In [14]:
root <- pw %>%
  filter(id %in% links$from & !id %in% links$to) %>%
  left_join(sets)
names(root) <- c("root_id", "root_name", "root_genes")
root_links <- links %>%
  filter(from %in% root$root_id)

Joining with `by = join_by(id)`


In [15]:
root$root_name

[1] "Autophagy"                           
 [2] "Cell Cycle"                          
 [3] "Cell-Cell communication"             
 [4] "Cellular responses to stimuli"       
 [5] "Chromatin organization"              
 [6] "Circadian Clock"                     
 [7] "DNA Repair"                          
 [8] "DNA Replication"                     
 [9] "Developmental Biology"               
[10] "Digestion and absorption"            
[11] "Disease"                             
[12] "Drug ADME"                           
[13] "Extracellular matrix organization"   
[14] "Gene expression (Transcription)"     
[15] "Hemostasis"                          
[16] "Immune System"                       
[17] "Metabolism"                          
[18] "Metabolism of RNA"                   
[19] "Metabolism of proteins"              
[20] "Muscle contraction"                  
[21] "Neuronal System"                     
[22] "Organelle biogenesis and maintenance"
[23] "Programmed Cell Death"               
[24] "Protein localization"                
[25] "Reproduction"                        
[26] "Sensory Perception"                  
[27] "Signal Transduction"                 
[28] "Transport of small molecules"        
[29] "Vesicle-mediated transport"

In [16]:
nrow(root)

[1] 29

## Identify sub-pathways

For our analysis, we'll use sub-pathways that are up to 4 levels below the Root nodes. For each level, we assemble the gene set and keep track of parent gene sets.

## Level 1

Just below the top nodes

In [17]:
root_split <- split(root_links, root_links$from)
l1 <- map2_dfr(
    root_split, names(root_split),
    function(link_df, parent_id) {
        parent_pw <- root %>%
          filter(root_id == parent_id)
        l1_pw <- pw %>%
          filter(id %in% link_df$to)
        names(l1_pw) <- c("l1_id", "l1_name")
        l1_pw <- l1_pw %>%
          mutate(root_id = parent_id) %>%
          left_join(root, by = "root_id")
        l1_pw
    }
)

In [18]:
l1 <- l1 %>%
  left_join(sets, by = c("l1_id" = "id"))

In [19]:
names(l1)[length(l1)] <- "l1_genes"

In [20]:
l1 <- l1 %>%
  filter(map_int(l1_genes, length) > 10)

In [21]:
l1_links <- links %>%
  filter(from %in% l1$l1_id)

In [22]:
nrow(l1)

[1] 153

## Level 2
Children of Level 1 nodes

In [23]:
l1_split <- split(l1_links, l1_links$from)
l2 <- map2_dfr(
    l1_split, names(l1_split),
    function(link_df, parent_id) {
        parent_pw <- l1 %>%
          filter(l1_id == parent_id)
        l2_pw <- pw %>%
          filter(id %in% link_df$to)
        names(l2_pw) <- c("l2_id", "l2_name")
        l2_pw <- l2_pw %>%
          mutate(l1_id = parent_id) %>%
          left_join(parent_pw, by = "l1_id")
        l2_pw
    }
)

Warning message in left_join(., parent_pw, by = "l1_id"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [24]:
l2 <- l2 %>%
  left_join(sets, by = c("l2_id" = "id"))

In [25]:
names(l2)[length(l2)] <- "l2_genes"

In [26]:
l2 <- l2 %>%
  filter(map_int(l2_genes, length) > 10)

In [27]:
l2_links <- links %>%
  filter(from %in% l2$l2_id)

In [28]:
nrow(l2)

[1] 358

## Level 3
Children of Level 2 nodes

In [29]:
l2_split <- split(l2_links, l2_links$from)
l3 <- map2_dfr(
    l2_split, names(l2_split),
    function(link_df, parent_id) {
        parent_pw <- l2 %>%
          filter(l2_id == parent_id)
        l3_pw <- pw %>%
          filter(id %in% link_df$to)
        names(l3_pw) <- c("l3_id", "l3_name")
        l3_pw <- l3_pw %>%
          mutate(l2_id = parent_id) %>%
          left_join(parent_pw, by = "l2_id")
        l3_pw
    }
)

In [30]:
l3 <- l3 %>%
  left_join(sets, by = c("l3_id" = "id"))
names(l3)[length(l3)] <- "l3_genes"

In [31]:
l3 <- l3 %>%
  filter(map_int(l3_genes, length) > 10)

In [32]:
nrow(l3)

[1] 465

In [33]:
l3_links <- links %>%
  filter(from %in% l3$l3_id)

## Level 4
Children of Level 3 nodes

In [34]:
l3_split <- split(l3_links, l3_links$from)
l4 <- map2_dfr(
    l3_split, names(l3_split),
    function(link_df, parent_id) {
        parent_pw <- l3 %>%
          filter(l3_id == parent_id)
        l4_pw <- pw %>%
          filter(id %in% link_df$to)
        names(l4_pw) <- c("l4_id", "l4_name")
        l4_pw <- l4_pw %>%
          mutate(l3_id = parent_id) %>%
          left_join(parent_pw, by = "l3_id")
        l4_pw
    }
)

Warning message in left_join(., parent_pw, by = "l3_id"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


In [35]:
l4 <- l4 %>%
  left_join(sets, by = c("l4_id" = "id"))
names(l4)[length(l4)] <- "l4_genes"

In [36]:
l4 <- l4 %>%
  filter(map_int(l4_genes, length) > 10)

In [37]:
nrow(l4)

[1] 378

## Assemble and output gene sets

Now that we've built out the gene sets from each level, we'll assemble these all in a list of gene sets that can be used for GSEA analysis.

We'll save the list of gene sets along with the information from each level, which will be used later for visualizaiton of the GSEA results.

In [38]:
set_list <- c(
    root$genes,
    l1$l1_genes,
    l2$l2_genes,
    l3$l3_genes,
    l4$l4_genes
)

names(set_list) <- c(
    root$id,
    l1$l1_id,
    l2$l2_id,
    l3$l3_id,
    l4$l4_id
)

all_sets <- list(
    root_tb = root,
    l1_tb = l1,
    l2_tb = l2,
    l3_tb = l3,
    l4_tb = l4,
    set_list = set_list
)

In [39]:
dir.create("output")

Warning message in dir.create("output"):
“'output' already exists”


In [40]:
saveRDS(
    all_sets, 
    paste0("output/reactome_gene_sets_", Sys.Date(), ".rds")
)

In [41]:
reactome_df <- data.frame(
    pathway = names(all_sets$set_list),
    n_pathway_genes = map_int(all_sets$set_list, length),
    pathway_genes = map_chr(all_sets$set_list, paste, collapse = ";")
)

In [42]:
nrow(reactome_df)

[1] 1354

In [43]:
reactome_df <- unique(reactome_df)

## Retrieve files

Now, we'll use the HISE SDK package to retrieve the MAST DEG results file based on its UUID. This will be placed in the `cache/` subdirectory by default.

In [44]:
file_uuid <- list(
    "fc83b89f-fd26-43b8-ac91-29c539703a45"
)

In [45]:
fres <- cacheFiles(file_uuid)

submitting request as query ID first...

retrieving files using fileIDS...



### Prepare DEG lists

To rank genes, we'll convert nomP to -log10(nomP), and incorporate the direction of differential expression by multiplying by the direction of effect size (sign(logFC)).

In [46]:
all_deg <- read.csv("cache/fc83b89f-fd26-43b8-ac91-29c539703a45/all_mast_deg_2023-09-06.csv")
all_deg$treat_time_type <- paste0(
    all_deg$fg, "_", 
    all_deg$timepoint, "_", 
    all_deg$aifi_cell_type)

Prior to ranking, we'll need to resolve missing `logFC` values. These can occur if one of the groups used for DEG analysis had no expression of the gene.

In [47]:
all_deg %>%
  filter(is.na(logFC)) %>%
  head()

,aifi_cell_type,timepoint,fg,bg,n_sample,gene,coef_C,coef_D,logFC,nomP,adjP,treat_time_type
,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,t_cd4_em,4,bortezomib,dmso,180,TFDP1,NA,3.196997,NA,3.817725e-05,0.1481468,bortezomib_4_t_cd4_em
2,t_cd4_treg,4,bortezomib,dmso,78,ABCA3,NA,-2.453613,NA,1.337154e-02,0.9999222,bortezomib_4_t_cd4_treg
3,t_cd4_treg,4,bortezomib,dmso,78,AC005070.3,NA,-2.259963,NA,3.110707e-02,0.9999222,bortezomib_4_t_cd4_treg
4,t_cd4_treg,4,bortezomib,dmso,78,AC006504.5,NA,-2.470421,NA,1.243915e-02,0.9999222,bortezomib_4_t_cd4_treg
5,t_cd4_treg,4,bortezomib,dmso,78,AC007686.3,NA,-2.265379,NA,3.041017e-02,0.9999222,bortezomib_4_t_cd4_treg
6,t_cd4_treg,4,bortezomib,dmso,78,AC010754.1,NA,-2.272969,NA,2.952072e-02,0.9999222,bortezomib_4_t_cd4_treg


When this occurs, we can use the sign of `coef_D` to determine the direction of expression change, rather than using the missing `logFC` value.

In [48]:
all_deg <- all_deg %>%
  mutate(direction = ifelse(
      is.na(logFC),
      sign(coef_D), # if missing logFC, use coef_D
      sign(logFC) # otherwise, use logFC
  ))

We also need to avoid nomP values of 0. These will cause NA values due to log transformation. We'll convert these to `1e-300` so that they have a non-zero value.

In [49]:
all_deg <- all_deg %>%
  mutate(nomP = ifelse(
      nomP == 0,
      1e-300, # if zero, change to 1e-300
      nomP # otherwise, keep the value
  ))

In [50]:
deg_list <- split(all_deg, all_deg$treat_time_type)

In [51]:
deg_list <- map(
    deg_list,
    function(deg) {
        deg %>%
          mutate(rank_val = -log10(nomP) * direction) %>%
          arrange(desc(rank_val))
    }
)

In [52]:
rank_list <- map(
    deg_list,
    function(deg) {
        v <- deg$rank_val
        names(v) <- deg$gene
        v
    }
)

## Run GSEA

In [53]:
parallel_param <- BiocParallel::MulticoreParam(
    workers = 4, 
    progressbar = FALSE
)

In [54]:
fgsea_res <- map(
    rank_list,
    function(ranks) {
        fgsea(
            pathways = all_sets$set_list,
            stats    = ranks,
            minSize  = 10,
            maxSize  = 500,
            BPPARAM  = parallel_param
        )
    }
)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (0.06% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


### Format results

In [55]:
deg_meta <- map(
    deg_list,
    function(deg) {
        list(
            fg = deg$fg[1],
            bg = deg$bg[1],
            timepoint = deg$timepoint[1],
            aifi_cell_type = deg$aifi_cell_type[1]
        )
    }
)

In [56]:
formatted_fgsea_res <- map2_dfr(
    fgsea_res,
    deg_meta,
    function(res, meta) {
        res %>%
          mutate(
              leadingEdge = map_chr(leadingEdge, paste, collapse = ";"),
              fg = meta$fg,
              bg = meta$bg,
              timepoint = meta$timepoint,
              aifi_cell_type = meta$aifi_cell_type
          ) %>%
          left_join(reactome_df, by = "pathway") %>%
          rename(id = pathway,
                 nomP = pval,
                 adjP = padj,
                 n_leadingEdge = size) %>%
          left_join(pw, by = "id") %>%
          rename(pathway = name) %>%
          select(fg, bg, timepoint, aifi_cell_type,
                 id, pathway, NES, nomP, adjP, 
                 n_leadingEdge, n_pathway_genes,
                 leadingEdge, pathway_genes) %>%
          arrange(desc(NES))

    }
)

In [57]:
head(formatted_fgsea_res)

fg,bg,timepoint,aifi_cell_type,id,pathway,NES,nomP,adjP,n_leadingEdge,n_pathway_genes,leadingEdge,pathway_genes
<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>
bortezomib,dmso,24,t_cd4_cm,R-HSA-8951664,Neddylation,2.371437,4.469096e-22,4.446751e-19,168,247,PSMD1;PSMD14;VCP;UBC;PSMD11;NPLOC4;PSMB2;PSMD7;PSMD12;PSMD2;PSMA6;PSMB1;PSMD4;PSMC6;PSMC1;SQSTM1;PSMB7;PSMC2;PSMD3;PSMB3;PSMB4;PSMA7;PSMA3;PSMD6;PSME4;PSMA5;PSMD13;PSMB6;SEM1;PSMC4;PSMA4;PSMA2;PSMB5;UBB;PSMC3;PSMA1;NFE2L2;PSMC5;PSMD8,AMER1;ANKRD9;ASB1;ASB10;ASB11;ASB12;ASB13;ASB14;ASB15;ASB16;ASB17;ASB18;ASB2;ASB3;ASB4;ASB5;ASB6;ASB7;ASB8;ASB9;BIRC5;BRCA1;BTBD1;BTBD6;BTRC;CAND1;CCDC22;CCDC8;CCNF;CDKN1A;CISH;COMMD1;COMMD10;COMMD2;COMMD3;COMMD4;COMMD5;COMMD6;COMMD7;COMMD8;COMMD9;COPS2;COPS3;COPS4;COPS5;COPS6;COPS7A;COPS7B;COPS8;CUL1;CUL2;CUL3;CUL4A;CUL4B;CUL5;CUL7;CUL9;DCAF10;DCAF11;DCAF13;DCAF16;DCAF17;DCAF4;DCAF5;DCAF6;DCAF7;DCAF8;DCUN1D1;DCUN1D2;DCUN1D3;DCUN1D4;DCUN1D5;DDA1;DDB1;DDB2;DPP3;DTL;ELOB;ELOC;EPAS1;ERCC8;FBXL12;FBXL13;FBXL14;FBXL15;FBXL16;FBXL18;FBXL19;FBXL20;FBXL21P;FBXL22;FBXL3;FBXL4;FBXL5;FBXL7;FBXL8;FBXO10;FBXO11;FBXO15;FBXO17;FBXO2;FBXO21;FBXO22;FBXO27;FBXO30;FBXO31;FBXO32;FBXO4;FBXO40;FBXO41;FBXO44;FBXO6;FBXO7;FBXO9;FBXW10;FBXW11;FBXW12;FBXW2;FBXW4;FBXW5;FBXW7;FBXW8;FBXW9;FEM1A;FEM1B;FEM1C;GAN;GPS1;HIF1A;HIF3A;KBTBD13;KBTBD6;KBTBD7;KBTBD8;KCTD6;KCTD7;KEAP1;KLHL11;KLHL13;KLHL2;KLHL20;KLHL21;KLHL22;KLHL25;KLHL3;KLHL41;KLHL42;KLHL5;KLHL9;LMO7;LRR1;LRRC41;MUL1;NAE1;NEDD8;NEURL2;NFE2L2;NPLOC4;NUB1;OBSL1;PALB2;PSMA1;PSMA2;PSMA3;PSMA4;PSMA5;PSMA6;PSMA7;PSMA8;PSMB1;PSMB10;PSMB11;PSMB2;PSMB3;PSMB4;PSMB5;PSMB6;PSMB7;PSMB8;PSMB9;PSMC1;PSMC2;PSMC3;PSMC4;PSMC5;PSMC6;PSMD1;PSMD10;PSMD11;PSMD12;PSMD13;PSMD14;PSMD2;PSMD3;PSMD4;PSMD5;PSMD6;PSMD7;PSMD8;PSMD9;PSME1;PSME2;PSME3;PSME4;PSMF1;PUM2;RBBP5;RBBP7;RBX1;RFWD2;RNF7;RPS27A;SEM1;SENP8;SKP1;SKP2;SOCS2;SOCS3;SOCS5;SOCS6;SPSB1;SPSB2;SPSB3;SPSB4;SQSTM1;TULP4;UBA3;UBA52;UBB;UBC;UBD;UBE2D1;UBE2D2;UBE2D3;UBE2F;UBE2M;UBXN7;UCHL3;UFD1L;VCP;VHL;WDR5;WDTC1;WSB1;WSB2;X;ZBTB16
bortezomib,dmso,24,t_cd4_cm,R-HSA-5619115,Disorders of transmembrane transporters,2.366473,3.186736e-19,6.341605e-17,106,177,PSMD1;PSMD14;VCP;UBC;PSMD11;PSMB2;PSMD7;PSMD12;PSMD2;PSMA6;PSMB1;PSMD4;PSMC6;PSMC1;PSMB7;PSMC2;PSMD3;PSMB3;PSMB4;PSMA7;PSMA3;PSMD6;PSME4;PSMA5;PSMD13;PSMB6;SEM1;PSMC4;PSMA4;PSMA2;NUP153;PSMB5;UBB;PSMC3;RANBP2;PSMA1;NUP205;PSMC5;PSMD8,AAAS;ABCA1;ABCA12;ABCA3;ABCB11;ABCB4;ABCB6;ABCC2;ABCC6;ABCC8;ABCC9;ABCD1;ABCD4;ABCG5;ABCG8;APOA1;AVP;AVPR1A;AVPR1B;AVPR2;BSG;CFTR;CP;DERL1;DERL2;DERL3;ERLEC1;ERLIN1;ERLIN2;GCK;GCKR;HEPH;HK1;KCNJ11;LMBRD1;NDC1;NUP107;NUP133;NUP153;NUP155;NUP160;NUP188;NUP205;NUP210;NUP214;NUP35;NUP37;NUP42;NUP43;NUP50;NUP54;NUP58;NUP62;NUP85;NUP88;NUP93;NUP98;OS9;POM121;POM121C;PSMA1;PSMA2;PSMA3;PSMA4;PSMA5;PSMA6;PSMA7;PSMA8;PSMB1;PSMB10;PSMB11;PSMB2;PSMB3;PSMB4;PSMB5;PSMB6;PSMB7;PSMB8;PSMB9;PSMC1;PSMC2;PSMC3;PSMC4;PSMC5;PSMC6;PSMD1;PSMD10;PSMD11;PSMD12;PSMD13;PSMD14;PSMD2;PSMD3;PSMD4;PSMD5;PSMD6;PSMD7;PSMD8;PSMD9;PSME1;PSME2;PSME3;PSME4;PSMF1;RAE1;RANBP2;RHAG;RNF185;RNF5;RPS27A;SEC13;SEH1L;SEL1L;SEM1;SLC11A2;SLC12A1;SLC12A3;SLC12A6;SLC16A1;SLC17A5;SLC17A8;SLC1A1;SLC1A3;SLC20A2;SLC22A12;SLC22A18;SLC22A5;SLC24A1;SLC24A4;SLC24A5;SLC26A2;SLC26A3;SLC26A4;SLC27A4;SLC29A3;SLC2A1;SLC2A10;SLC2A2;SLC2A9;SLC33A1;SLC34A1;SLC34A2;SLC34A3;SLC35A1;SLC35A2;SLC35A3;SLC35C1;SLC36A2;SLC39A4;SLC3A1;SLC3A2;SLC40A1;SLC4A1;SLC4A4;SLC5A1;SLC5A2;SLC5A5;SLC5A7;SLC6A14;SLC6A18;SLC6A19;SLC6A2;SLC6A20;SLC6A3;SLC6A5;SLC7A7;SLC7A9;SLC9A6;SLC9A9;SLCO1B1;SLCO1B3;SLCO2A1;TPR;UBA52;UBB;UBC;VCP
bortezomib,dmso,24,t_cd4_cm,R-HSA-6807070,PTEN Regulation,2.345639,5.914341e-18,7.355961e-16,115,158,PSMD1;PSMD14;UBC;PSMD11;PSMB2;PSMD7;PSMD12;PSMD2;PSMA6;PSMB1;PSMD4;PSMC6;PSMC1;PSMB7;PSMC2;PSMD3;PSMB3;TNKS;PSMB4;PSMA7;PSMA3;PSMD6;PSME4;PSMA5;PSMD13;PSMB6;SEM1;PSMC4;PSMA4;PSMA2;RHEB;PSMB5;UBB;PSMC3;RPTOR;PSMA1;PSMC5;PSMD8,AGO1;AGO2;AGO3;AGO4;AKT1;AKT2;AKT3;ATF2;ATN1;BMI1;CBX2;CBX4;CBX6;CBX8;CHD3;CHD4;CNOT6L;CSNK2A1;CSNK2A2;CSNK2B;EED;EGR1;EZ

## Write output file

Write the metadata as a .csv for later use. We remove `row.names` and set `quote = FALSE` to simplify the outputs and increase compatibility with other tools.

In [58]:
write.csv(
    formatted_fgsea_res,
    paste0("output/all_reactome_gsea_res_", Sys.Date(), ".csv"),
    row.names = FALSE,
    quote = FALSE
)

## Store results in HISE

Finally, we store the output file in our Collaboration Space for later retrieval and use. We need to provide the UUID for our Collaboration Space (aka `studySpaceId`), as well as a title for this step in our analysis process.

The hise function `uploadFiles()` also requires the FileIDs from the original fileset for reference, which we used above when the DEG results were retrieved (`file_uuid`)

In [59]:
study_space_uuid <- "40df6403-29f0-4b45-ab7d-f46d420c422e"
title <- paste("VRd TEA-seq Reactome GSEA Analysis", Sys.Date())

In [60]:
out_files <- list.files(
    "output",
    full.names = TRUE
)
out_list <- as.list(out_files)

In [61]:
uploadFiles(
    files = out_list,
    studySpaceId = study_space_uuid,
    title = title,
    inputFileIds = file_uuid,
    store = "project",
    doPrompt = FALSE
)

$files
$files[[1]]
[1] "output/all_reactome_gsea_res_2023-09-06.csv"

$files[[2]]
[1] "output/reactome_gene_sets_2023-09-06.rds"


$traceId
[1] "a693d09a-dbd2-4771-9841-586778a61b48"

In [62]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.23.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] fgsea_1.26.0 tibble_3.2.1 dplyr_1.1.2  purrr_1.0.2  hise_2.16.0 

loaded via a namespace (and not attached):
 [1] Matrix_1.6-1        gtable_0.3.3        jsonlite_1.8.7     
 [4] compiler_4.3.1      crayon_1.5.2        tidyselect_1.2.0   
 [7] Rcpp_1.0.11         IRdisplay_1.1       bitops_1.0-7       
[10] 